# Using a Modflow UZF model as a stressmodel in Pastas

In [2]:
import os

import flopy
import pandas as pd
import pastas as ps

import pastas_plugins.modflow as ppmf

from pastas.timer import SolveTimer

ps.set_log_level("ERROR")

bindir = "bin"
mf6_exe = os.path.join(bindir, "mf6")
if not os.path.isfile(mf6_exe):
    if not os.path.isdir(bindir):
        os.makedirs(bindir)
    flopy.utils.get_modflow("bin", repo="modflow6")

In [3]:
pd.read_csv("data/aftopping.csv", index_col=0, parse_dates=True)

,B28H1808_2,Precipitation,Evaporation
2002-06-10,NaN,0.002687,0.003967
2002-06-11,NaN,0.006737,0.002562
2002-06-12,NaN,0.008575,0.002883
2002-06-13,NaN,0.009862,0.002133
2002-06-14,NaN,0.005000,0.001046
...,...,...,...
2017-01-28,20.00,0.000000,0.000504
2017-01-29,20.00,0.000875,0.000500
2017-01-30,20.04,0.004275,0.000308
2017-01-31,20.10,0.003812,0.000108
